In [1]:
import numpy as np
import pandas as pd
from hydra import initialize, compose
import dill
import os
import logging
import time
import matplotlib.pyplot as plt
plt.style.use('default')

import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler
from qiskit.primitives import BackendSampler
from qiskit_aer import AerSimulator
from qiskit_aer import AerProvider

from os.path import dirname, abspath
import sys
parent_dir = dirname(dirname(abspath(' ')))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from src.bitflip_strategy.Local_Search_PUBO import local_search, comp_objective
from src.optimizer.Quantum_Optimizer_FGSR_b_hardware import QuantumOptimizer
from src.optimizer.utils_logger import make_logger, initialize_data
from src.optimizer.hardware_utils import create_ansatz, get_initial_layout
from Cplex_HuboSolver import CPLEX_HUBOsolver

import warnings
warnings.simplefilter('ignore')

In [2]:
# Load config file
with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name='config_sample.yaml')

In [3]:
# Set backend
env = cfg.backend.env
qiskit_version = int(qiskit.__version__.split(".")[0])

if cfg.backend.env == "hardware" or cfg.layout.use_real_backend:
    service = QiskitRuntimeService(**cfg.QiskitRuntimeService)

if cfg.layout.use_real_backend:
    layout_backend = service.get_backend(cfg.layout.name)
else:
    layout_backend = None

if cfg.backend.env == "hardware":
    backend = service.get_backend(cfg.backend.name)
    layout_backend = backend
else:
    backend = AerProvider().get_backend(cfg.backend.name)
    service = None

In [4]:
backend

AerSimulator('aer_simulator_matrix_product_state')

In [5]:
layout_backend

<IBMBackend('ibm_torino')>

In [6]:
# Initialize data and logger
output_dir, idx_set, theta, x_ini, fval, indval, df_job_id, df_result = initialize_data(cfg)
logger = make_logger(cfg, output_dir, backend, env)

Parameters:
	instance:
		dir = ../instances/SpinGlass
		list = ['ibmq_guadalupe_0', 'ibm_geneva_0', 'ibm_washington_5']
		trial_list = [0]
	optimizer:
		shots = 10000
		search = True
		n_flips = 50
		n_reps = 2
		alpha = 0.4
		random_update = True
		max_epoch = 1
		restart_s = 0.8
		restart_std = 0.0
		early_stop = False
		mixing_ratio = 0
		refval = None
		intermediate_result_display = True
		save_quasi_dist = False
	options:
		save_result = True
		output_dir = results
		timezone = Asia/Tokyo
		verbose = True
		verbose_optimization = False
		restart = False
		date = None
	backend:
		env = simulator
		name = aer_simulator_matrix_product_state
	layout:
		use_real_backend = True
		name = ibm_torino
		automatic_selection = True
		start_qubit = 14
	QiskitRuntimeService:
		channel = ibm_quantum
		instance = client-enablement/its/its-skill-buildi
	runtime:
		resilience_level = 0
		optimization_level = 0
		skip_transpilation = True
	transpile:
		optimization_level = 3


output_dir: results/20240817-182610


In [7]:
idx_set

[]

In [8]:
# Run optimization
logger.info("Backend Info:")
logger.info(f"\tbackend = {backend}")
logger.info(f"\tlayout_backend = {layout_backend}")
instance_list = cfg.instance.list
logger.info("Problem Info:")
logger.info(f"\tinstance_list = {instance_list}")
shots = cfg.optimizer.shots

result_data = []
for instance in instance_list:
    # Load instance data
    with open(f'{cfg.instance.dir}/{instance}_hubo.pickle', mode='br') as fi:
        data = dill.load(fi)

    # Convert data to pubo format
    pubo_terms = {}
    offset = 0
    qubits_set = set()
    ord = -1
    for key, value in data.items():
        if key == -1:
            offset = value
            continue
        pubo_terms[key] = value
        ord = max(ord, len(key))
        for i in key:
            qubits_set.add(i)
    num_qubits = len(qubits_set)
    obj_data = [num_qubits, pubo_terms]
    cplex_result = CPLEX_HUBOsolver().solve(obj_data)
    refval = cplex_result.fval + offset

    pubo_data = []
    for key in data.keys():
        if key == -1:
            pubo_offset = data[key]
        else:
            l = list(key)
            i = ord - len(l)
            if i > 0:
                for _ in range(i):
                    l.append(l[-1])
            l.append(data[key])
            pubo_data.append(l)
    pubo_data = np.asarray(pubo_data, dtype=float)

    # Define objective
    objective = lambda x: comp_objective(x, ord, pubo_data, pubo_offset)
    def search(x):
        x = local_search(x, ord, pubo_data, pubo_offset, cfg.optimizer.n_flips)
        return np.asarray(x, dtype=int)

    logger.info("-"*130)
    logger.info(f"instance = {instance} \t num_qubits = {num_qubits} \t refval = {refval}")
    logger.info("-"*130)

    for trial in cfg.instance.trial_list:
        np.random.seed(trial)
        if output_dir is not None:
            output_dir_pb_trial = f"{output_dir}/{instance}_trial{trial}"
            if os.path.exists(output_dir_pb_trial) == False:
                os.mkdir(output_dir_pb_trial)
            with open(f"{output_dir_pb_trial}/pubo_data.pickle", 'wb') as f:
                dill.dump(pubo_data, f)
        logger_pb = logging.getLogger(f"SpinGlass Optimization Log: (instance, shots, trial) = {(instance, shots, trial)}")
        logger_pb.setLevel(logging.DEBUG)
        if cfg.options.verbose_optimization:
            st_handler_pb = logging.StreamHandler()
            st_handler_pb.setLevel(logging.DEBUG)
            logger_pb.addHandler(st_handler_pb)
        if output_dir is not None:
                fl_handler_pb = logging.FileHandler(filename=f"{output_dir_pb_trial}/optimization.log", mode="a")
                fl_handler_pb.setLevel(logging.DEBUG)
                logger_pb.addHandler(fl_handler_pb)
        logger_pb.info(f"instance: {instance} \t refval = {refval:.3f} \t shots = {shots} \t trial = {trial}")
        
        # create initial_layout
        chain_nodes = [i for i in range(num_qubits)]
        sep_nodes = []
        sep_nodes_coupling = []
        if layout_backend is not None:
            if cfg.layout.automatic_selection:
                chain_nodes, sep_nodes, sep_nodes_coupling = get_initial_layout(layout_backend, num_qubits, cfg.layout.start_qubit)
                initial_layout = chain_nodes + sep_nodes
            else:
                initial_layout = cfg.initial_layout[cfg.layout.name][:num_qubits]
        else:
            initial_layout = None
        layout_data = {
            "initial_layout": initial_layout,
            "chain_nodes": chain_nodes,
            "sep_nodes": sep_nodes,
            "sep_nodes_coupling": sep_nodes_coupling
        }
        if output_dir is not None:
            with open(f"{output_dir_pb_trial}/layout_data.pickle", 'wb') as f:
                dill.dump(layout_data, f)
        sub_qc = create_ansatz(cfg.optimizer.n_reps, chain_nodes, sep_nodes, sep_nodes_coupling)
        
        with Session(service=service, backend=backend) as session:
            if env == "hardware":
                options = Options()
                options.execution.shots = shots
                #options.optimization_level = cfg.runtime.optimization_level
                options.resilience_level = cfg.runtime.resilience_level
                options.transpilation.skip_transpilation = cfg.runtime.skip_transpilation # True
                if cfg.backend.name not in ["simulator_stabilizer", "simulator_mps", "simulator_extended_stabilizer", "ibmq_qasm_simulator", "simulator_statevector"]:
                    options.transpilation.initial_layout = initial_layout
                sampler = Sampler(session=session, options=options)
            else:
                sampler = BackendSampler(backend=backend)
                sampler.set_options(shots=shots, seed_simulator=trial)
            QuantumSolver = QuantumOptimizer()
            QuantumSolver.set_optimizer(num_qubits, sub_qc, sampler, env, backend, cfg.transpile.optimization_level, initial_layout)
            QuantumSolver.run(
                objective = objective,
                search = search,
                alpha = cfg.optimizer.alpha,
                random_update = cfg.optimizer.random_update,
                max_epoch = cfg.optimizer.max_epoch,
                restart_s = cfg.optimizer.restart_s,
                restart_std = cfg.optimizer.restart_std,
                refval = refval,
                intermediate_result_display = cfg.optimizer.intermediate_result_display,
                early_stop = cfg.optimizer.early_stop,
                x_ini = x_ini,
                output_dir = output_dir_pb_trial,
                logger = logger_pb,
                seed = trial,
                idx_set = idx_set,
                df_job_id = df_job_id,
                df_result = df_result,
                theta = theta,
                mixing_ratio=cfg.optimizer.mixing_ratio,
                fval=fval,
                indval=indval,
                save_quasi_dist=cfg.optimizer.save_quasi_dist
            )
        step = len(QuantumSolver.result_history)
        objval = QuantumSolver.result_history[-1][3]
        expval = QuantumSolver.result_history[-1][4]
        sprob = QuantumSolver.result_history[-1][5]
        result_data.append([instance, num_qubits, refval, backend.name, shots, trial, step, objval, expval, sprob])
        time.sleep(5)
        logger.info(f"trial = {trial} \t step = {step} \t objval = {objval} \t cvar = {expval} \t s-prob = {sprob}")
        df = pd.DataFrame(data=result_data, columns=["instance", "num_qubits", "refval", "backend", "shots", "trial", "step", "objval", "cvar", "s-prob"])
        df.to_csv(f"{output_dir}/df_result_summary.csv", index=False)

Backend Info:
	backend = aer_simulator_matrix_product_state
	layout_backend = <IBMBackend('ibm_torino')>
Problem Info:
	instance_list = ['ibmq_guadalupe_0', 'ibm_geneva_0', 'ibm_washington_5']
----------------------------------------------------------------------------------------------------------------------------------
instance = ibmq_guadalupe_0 	 num_qubits = 16 	 refval = -22.0
----------------------------------------------------------------------------------------------------------------------------------


	epoch = 0 	 step = 0 	 (j, k) = (0, 11) 	 objval = -22.000 	 cvar = -5.702   	 s-prob = 2.000e-04
	epoch = 0 	 step = 1 	 (j, k) = (0, 15) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 6.680e-01
	epoch = 0 	 step = 2 	 (j, k) = (1, 24) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 6.680e-01
	epoch = 0 	 step = 3 	 (j, k) = (2, 30) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 7.359e-01
	epoch = 0 	 step = 4 	 (j, k) = (3, 10) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 7.359e-01
	epoch = 0 	 step = 5 	 (j, k) = (4, 12) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 7.359e-01
	epoch = 0 	 step = 6 	 (j, k) = (5, 2) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.059e-01
	epoch = 0 	 step = 7 	 (j, k) = (6, 18) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.059e-01
	epoch = 0 	 step = 8 	 (j, k) = (7, 19) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.059e-01
	epoch = 0 	 step = 9 	 (j, k) = (8, 17) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.057e-01
	e

trial = 0 	 step = 33 	 objval = 4 	 cvar = -22.0 	 s-prob = -22.0
----------------------------------------------------------------------------------------------------------------------------------
instance = ibm_geneva_0 	 num_qubits = 27 	 refval = -42.0
----------------------------------------------------------------------------------------------------------------------------------


	epoch = 0 	 step = 0 	 (j, k) = (0, 53) 	 objval = -42.000 	 cvar = -7.786   	 s-prob = 0.000e+00
	epoch = 0 	 step = 1 	 (j, k) = (0, 4) 	 objval = -42.000 	 cvar = -40.465   	 s-prob = 2.549e-01
	epoch = 0 	 step = 2 	 (j, k) = (1, 35) 	 objval = -42.000 	 cvar = -40.774   	 s-prob = 2.856e-01
	epoch = 0 	 step = 3 	 (j, k) = (2, 20) 	 objval = -42.000 	 cvar = -40.965   	 s-prob = 2.965e-01
	epoch = 0 	 step = 4 	 (j, k) = (3, 45) 	 objval = -42.000 	 cvar = -41.236   	 s-prob = 3.236e-01
	epoch = 0 	 step = 5 	 (j, k) = (4, 44) 	 objval = -42.000 	 cvar = -41.236   	 s-prob = 3.236e-01
	epoch = 0 	 step = 6 	 (j, k) = (5, 12) 	 objval = -42.000 	 cvar = -41.534   	 s-prob = 3.534e-01
	epoch = 0 	 step = 7 	 (j, k) = (6, 27) 	 objval = -42.000 	 cvar = -41.534   	 s-prob = 3.534e-01
	epoch = 0 	 step = 8 	 (j, k) = (7, 8) 	 objval = -42.000 	 cvar = -41.534   	 s-prob = 3.534e-01
	epoch = 0 	 step = 9 	 (j, k) = (8, 25) 	 objval = -42.000 	 cvar = -41.542   	 s-prob = 3.542e-01
	ep

trial = 0 	 step = 55 	 objval = 21 	 cvar = -42.0 	 s-prob = -42.0
----------------------------------------------------------------------------------------------------------------------------------
instance = ibm_washington_5 	 num_qubits = 127 	 refval = -198.0
----------------------------------------------------------------------------------------------------------------------------------


	epoch = 0 	 step = 0 	 (j, k) = (0, 158) 	 objval = -180.000 	 cvar = -17.764   	 s-prob = 0.000e+00
	epoch = 0 	 step = 1 	 (j, k) = (0, 248) 	 objval = -196.000 	 cvar = -166.307   	 s-prob = 0.000e+00
	epoch = 0 	 step = 2 	 (j, k) = (0, 15) 	 objval = -198.000 	 cvar = -179.194   	 s-prob = 0.000e+00
	epoch = 0 	 step = 3 	 (j, k) = (0, 122) 	 objval = -198.000 	 cvar = -181.158   	 s-prob = 4.500e-03
	epoch = 0 	 step = 4 	 (j, k) = (1, 78) 	 objval = -198.000 	 cvar = -181.232   	 s-prob = 4.700e-03
	epoch = 0 	 step = 5 	 (j, k) = (2, 65) 	 objval = -198.000 	 cvar = -181.232   	 s-prob = 4.700e-03
	epoch = 0 	 step = 6 	 (j, k) = (3, 132) 	 objval = -198.000 	 cvar = -181.232   	 s-prob = 4.700e-03
	epoch = 0 	 step = 7 	 (j, k) = (4, 27) 	 objval = -198.000 	 cvar = -181.680   	 s-prob = 4.900e-03
	epoch = 0 	 step = 8 	 (j, k) = (5, 31) 	 objval = -198.000 	 cvar = -181.888   	 s-prob = 5.200e-03
	epoch = 0 	 step = 9 	 (j, k) = (6, 84) 	 objval = -198.000 	 cvar = -182.360 

trial = 0 	 step = 257 	 objval = 19 	 cvar = -198.0 	 s-prob = -198.0
